In [1]:
from os.path import join
from keras._tf_keras.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from keras._tf_keras.keras.models import load_model

from src.hpe_dnn.architecture import DnnArch
from src.hpe_dnn.model import HpeDnn, HpeDnnConstructorArgs, HpeDnnModelInitializeArgs
from src.common.helpers import read_dataframe
from src.hpe_dnn.helpers import df_to_dataset

2025-09-10 13:16:16.213175: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


loaded labels


/mnt/c/Projects/climbing-technique-detector/.venv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [3]:

model = HpeDnn(args=HpeDnnConstructorArgs(
    name="arch1-mp-test",
    model_arch=DnnArch.ARCH1,
    dataset_name="techniques_mp"
))
model.initialize_model(args=HpeDnnModelInitializeArgs())

loading a fresh model 'DnnArch.ARCH1'
used numeric features:  ['NOSE_x', 'NOSE_y', 'NOSE_z', 'NOSE_visibility', 'LEFT_SHOULDER_x', 'LEFT_SHOULDER_y', 'LEFT_SHOULDER_z', 'LEFT_SHOULDER_visibility', 'LEFT_ELBOW_x', 'LEFT_ELBOW_y', 'LEFT_ELBOW_z', 'LEFT_ELBOW_visibility', 'LEFT_WRIST_x', 'LEFT_WRIST_y', 'LEFT_WRIST_z', 'LEFT_WRIST_visibility', 'RIGHT_SHOULDER_x', 'RIGHT_SHOULDER_y', 'RIGHT_SHOULDER_z', 'RIGHT_SHOULDER_visibility', 'RIGHT_ELBOW_x', 'RIGHT_ELBOW_y', 'RIGHT_ELBOW_z', 'RIGHT_ELBOW_visibility', 'RIGHT_WRIST_x', 'RIGHT_WRIST_y', 'RIGHT_WRIST_z', 'RIGHT_WRIST_visibility', 'LEFT_HIP_x', 'LEFT_HIP_y', 'LEFT_HIP_z', 'LEFT_HIP_visibility', 'LEFT_KNEE_x', 'LEFT_KNEE_y', 'LEFT_KNEE_z', 'LEFT_KNEE_visibility', 'LEFT_ANKLE_x', 'LEFT_ANKLE_y', 'LEFT_ANKLE_z', 'LEFT_ANKLE_visibility', 'LEFT_HEEL_x', 'LEFT_HEEL_y', 'LEFT_HEEL_z', 'LEFT_HEEL_visibility', 'LEFT_FOOT_INDEX_x', 'LEFT_FOOT_INDEX_y', 'LEFT_FOOT_INDEX_z', 'LEFT_FOOT_INDEX_visibility', 'RIGHT_HIP_x', 'RIGHT_HIP_y', 'RIGHT_HIP_z', 

2025-09-10 13:19:35.634586: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-10 13:19:35.977913: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-10 13:19:36.719883: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-10 13:19:38.155836: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-10 13:19:40.930853: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-10 13:19:48.682337: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-10 13:19:59.236522: I tensorflow/core/framework/local_rendezvous.cc:407] L

In [4]:

df_train = read_dataframe("data/df/techniques_mp/train.pkl")
ds_train = df_to_dataset(df_train, augment=False, balance=False, shuffle=True)

df_val = read_dataframe("data/df/techniques_mp/val.pkl")
ds_val = df_to_dataset(df_val, augment=False, balance=False, shuffle=False)

In [6]:

df_old = read_dataframe("data/df/techniques/train.pkl")
ds_old = df_to_dataset(df_old, augment=False, balance=False, shuffle=True)

KeyError: 'label'

In [ ]:
from src.hpe_dnn.helpers import demo_batch

#demo_batch(ds_train)
ds_train.element_spec

In [ ]:
keras.config.disable_traceback_filtering()

In [5]:
#checkpoint_path = join("data/runs/hpe_dnn/arch1-mp-test/train/models", "epoch_{epoch:02d}__val_accuracy_{val_categorical_accuracy:.4f}.keras")
checkpoint_path = join("data/runs/hpe_dnn/arch1-mp-test/train/models", "saved_model.keras")
cp_callback = ModelCheckpoint(checkpoint_path, 
    save_best_only=True, 
    save_weights_only=False, 
    verbose=0,
    monitor="val_categorical_accuracy",
    mode="max")


model.model.fit(ds_train, epochs=2, validation_data=ds_val,
    callbacks=[
        cp_callback
    ])

Epoch 1/2


2025-09-10 13:20:22.170137: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f7c7c03ab30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-10 13:20:22.170180: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2025-09-10 13:20:22.457957: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-10 13:20:23.497523: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 90300


 12/120 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - categorical_accuracy: 0.4996 - loss: 1.6600

I0000 00:00:1757503227.717556   25485 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - categorical_accuracy: 0.6167 - loss: 1.3743

TypeError: No conversion path for dtype: dtype('<U6')